# 1 Voronoi

In [1]:
import numpy as np
import matplotlib.pylab as plt
import time 
%matplotlib inline
import pickle
import scipy 
from scipy.spatial import Delaunay
from numpy import linalg as LA

In [11]:
with open('initialVariables.pickle', 'rb') as f:  # Python 3: open(..., 'rb')
    position, __ , heading = pickle.load(f)
with open('obsRotated.pickle', 'rb') as f:  # Python 3: open(..., 'rb')
    obstacles, __ = pickle.load(f)

In [77]:
points = np.array(obstacles[:,0:2])
p = position[0:2]
r = obstacles [:,3]

In [78]:
points

array([[ 0.5018485 , -2.17902457],
       [-0.90593594, -2.04432876],
       [-2.81556889,  0.26939163],
       [ 0.5018485 , -2.17902457],
       [-1.04063176, -3.4521132 ],
       [-1.54248026, -1.27308863],
       [-0.26939163, -2.81556889],
       [ 2.9502647 ,  1.13839281],
       [ 1.04063176,  3.4521132 ],
       [ 4.22335333, -0.40408745]])

In [79]:
#points = np.array([[-2, 0], [0, 1.1], [1, 0], [1, 1], [3,4]])

tri = Delaunay(points)

In [80]:
median = points[tri.simplices]

In [81]:
#p = np.array([0.1, 0.6])

In [82]:
r

array([ 2.,  1.,  2.,  1.,  1.,  2.,  2.,  2.,  1.,  1.])

In [83]:
triangle_index = tri.find_simplex(p)

In [84]:
tri.plane_distance(p)

array([ 1.89372404, -1.64879138, -0.14680222,  2.40438747,  2.51907612,
       -2.03237559, -1.89879344,  0.75385832, -0.83336034, -1.34792795,
       -1.28039329])

In [85]:
obstacles

array([[ 0.5018485 , -2.17902457,  0.        ,  2.        ],
       [-0.90593594, -2.04432876,  0.        ,  1.        ],
       [-2.81556889,  0.26939163,  0.        ,  2.        ],
       [ 0.5018485 , -2.17902457,  0.        ,  1.        ],
       [-1.04063176, -3.4521132 ,  0.        ,  1.        ],
       [-1.54248026, -1.27308863,  0.        ,  2.        ],
       [-0.26939163, -2.81556889,  0.        ,  2.        ],
       [ 2.9502647 ,  1.13839281,  0.        ,  2.        ],
       [ 1.04063176,  3.4521132 ,  0.        ,  1.        ],
       [ 4.22335333, -0.40408745,  0.        ,  1.        ]])

In [86]:
points_index = points[tri.simplices[triangle_index]]
#tri.simplices[triangle_index]
radius_index = r[tri.simplices[triangle_index]]
rads= np.hstack([radius_index, radius_index[0]])
base = np.vstack([points_index, points_index[0]])


In [96]:


# for i in range(len(points_index)):
#     median= (base[i]+base[i+1])/2
#     print (base[i], base[i+1], median)
    
median =np.array([ [(base[i]+base[i+1]), 
                    rads[i]-rads[i+1],
                    #norm new
                    LA.norm(np.add([base[i],base[i+1]],np.multiply([rads[i], -rads[i+1]],np.divide(([base[i+1]-base[i]]),LA.norm([base[i+1],base[i]]))))),
                    # R times V/V
                    np.multiply([rads[i], -rads[i+1]],np.divide(([base[i+1]-base[i]]),LA.norm([base[i+1],base[i]]))),
                    # Normalised vector
                    np.divide(([base[i+1]-base[i]]),LA.norm([base[i+1],base[i]])),
                    # Vector 
                    [base[i+1]-base[i]],
                    # New midpoint
                    np.divide(base[i]+base[i+1]+np.multiply((rads[i]-rads[i+1]),np.divide(([base[i+1]-base[i]]),LA.norm([base[i+1],base[i]]))),2),
                    LA.norm([base[i], base[i+1]])] 
                  for i in range(len(points_index)) ])
median

array([[array([ 1.40778444, -0.13469582]), 0.0, 5.9971576153283586,
        array([[ 2.40147319, -1.28899105]]),
        array([[ 1.2007366 ,  0.64449552]]),
        [array([ 4.49274496,  2.41148144])],
        array([[ 0.70389222, -0.06734791]]), 3.7416573867739413],
       [array([ 3.99089646,  4.59050601]), 1.0, 3.7347182337863067,
        array([[-0.79637199, -0.48244405]]),
        array([[-0.39818599,  0.48244405]]),
        [array([-1.90963294,  2.31372039])],
        array([[ 1.79635523,  2.53647503]]), 4.7958315233127191],
       [array([-0.5018485 ,  2.17902457]), -1.0, 6.2377713376166462,
        array([[-0.62649669,  2.29205956]]),
        array([[-0.62649669, -1.14602978]]),
        [array([-2.58311202, -4.72520183])],
        array([[ 0.06232409,  1.66252718]]), 4.1231056256176606]], dtype=object)

In [88]:
# np.amax(median, axis=1)
np.add([1,2],[3,4])

array([4, 6])

In [23]:

winner_index =median[np.argmax(median[:,1])]
dx_dy = winner_index[0]-p
psi = np.arctan(dx_dy[1]/dx_dy[0])
print(dx_dy, p,winner_index[0])
new_heading = psi-heading


[ 1.99544823  2.29525301] [0 0] [ 1.99544823  2.29525301]


In [ ]:
# import matplotlib.delaunay as triang
# import pylab
# import numpy
# from numpy import linalg as LA

# # 10 random points (x,y) in the plane
# x,y =  numpy.array(numpy.random.standard_normal((2,10)))
# cens,edg,tri,neig = triang.delaunay(x,y)

# for t in tri:
#  # t[0], t[1], t[2] are the points indexes of the triangle
#  t_i = [t[0], t[1], t[2], t[0]]
#  pylab.plot(x[t_i],y[t_i])

# pylab.plot(x,y,'o')
# pylab.show()
#print(x, y, tri, neig)